We put our import statements at the top of our script, because we're civilized.

In [1]:
from scipy.special import stdtr as t_cdf
import numpy as np
import pandas as pd
import scipy

from sklearn.datasets import load_iris

Then we load our data because we have data because we did the science. 

In [2]:
df = load_iris(as_frame = True)['data']
df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2


In [3]:
print('number of observations:', df.shape[0])

number of observations: 150


In [4]:
# get our data as vectors
pw = df['petal width (cm)'].to_numpy()
pl = df['petal length (cm)'].to_numpy()
sw = df['sepal width (cm)'].to_numpy()
sl = df['sepal length (cm)'].to_numpy()

Now, we'll do an analysis. For the sake of this example, we'll pretend we don't have access to a function that does _t_-tests. I know, it's unthinkable. 

We will write our _t_-test from scratch, like a common experimental design student. Let's find out if petals are longer than they are wide. 

In [5]:
# compute the paired-sample t statistic
t = (pl-pw).mean()/np.sqrt((((pl-pw)-(pl-pw).mean())**2).sum()/149/150)
# and then compute the p-value
p = t_cdf(149, -np.abs(t))*2
# print results
print('t = %f, p = %e'%(t, p))

t = 29.796754, p = 1.200807e-64


We also want to know whether sepals are wider than 3 cm (for some reason), so let's do that too.

In [6]:
# one sample t-test
t = (sw-3).mean()/np.sqrt((((sw-3)-(sw-3).mean())**2).sum()/149/150)
p = t_cdf(149, -t)
print('t = %f, p = %f'%(t, p))

t = 1.611015, p = 0.054646


These results do match what you'd get out of scipy's _t_-test function, so it is "correct" insofar as it did what you wanted. 

But it can still be improved. How?
1. Variable names aren't descriptive. This script is pretty short, so you can just look up and see what they mean, but that won't always be the case.
2. Lines of code are repeated almost verbatim, so if you ever change the code, you'll have to remember to change it in multiple places. (This makes bugs more likely to occur.)
3. Difficult to tell what's going on. (For instance, can you tell if these are one-tailed or two-tailed tests?)
3. Some values are hard coded, which makes the code difficult to re-use. 

Let's imagine, for example, that someone ran a replication study (this is, after all, a groundbreaking advance in botany). They'll have to comb through the code and find all the idiosyncratic things you decided to hard code into your script. You'd have to do the same thing if you wanted to re-use this code on another dataset, or your results would be incorrect. 

Let's modify things to make them a bit more human-friendly. We'll start with our one-sample test.

In [7]:
# if you hard code a variable, put it at the top of a chunk
# and MAKE IT NOTICEABLE with capitalization
NULL_MEAN = 3 

delta = sw - NULL_MEAN 
n_obs = delta.shape[0]
var = ((delta - delta.mean())**2).sum() / (n_obs - 1)
se = np.sqrt(var / n_obs) 
t = delta.mean() / se
p = t_cdf(n_obs - 1, -t)

print('t = %f, p = %f'%(t, p))

t = 1.611015, p = 0.054646


I think this is already a bit more readable. 
1. We broke up that awful line that computed the _t_-stat before into some smaller lines. This makes is easier to see what's going on. 
2. Hard-coded numbers have been remove. (`n` is now taken from `delta.shape`.) This doesn't just mean less editing later, but also makes it more clear what the value actually _means_.

Notice that we made it more clear what's going on without adding any comments. The best code is _self-documenting_; it should be relatively clear what's going on without comments. (Still comment your code, though, but more on that later.)

Still, we'd need to copy lines of code verbatim if we wanted to do the paired-sample test up above. Let's make our code a bit more reusable using the magic of _functions._

In [8]:
def compute_t_1samp(x, null_mean = 0):
    delta = x - null_mean 
    n_obs = delta.shape[0]
    var = ((delta - delta.mean())**2).sum() / (n_obs - 1)
    se = np.sqrt(var / n_obs) 
    t = delta.mean() / se
    return t

t = compute_t_1samp(sw, null_mean = 3)
p = t_cdf(sw.shape[0] - 1, -t)
print('t = %f, p = %f'%(t, p))

t = 1.611015, p = 0.054646


Note that I replaced, in our function definition, any reference to the data we're using (which we had called `sw`) with a generic variable name `x`. `x` is a placeholder for whatever I put into the function. It doesn't exist outside of the function. If I _try_ to access `x` outside of the function, it'll throw an error. If I name another variable `x` outside of the function, it won't affect how the function works. This is how things should be; everything is compartmentalized, which keeps the probability of changing one part of the code breaking another part of the code is low. 

Now, we have a re-useable function that can compute one-sample _t_-statistics for us. Since a paired-sample _t_-test is just a one-sample _t_-test on the paired differences, we can compute our paired _t_-statistic from above as follows.

In [9]:
compute_t_1samp(pl - pw) # no repeat code!

29.79675378021898

There's still something a bit hard to parse about the line where we compute the _p_ value. If you're not great at remembering formulas, it may be hard to parse whether we're computing a _p_ value for a one- or two-tailed test. (And in fact, we're doing two different types of test in the above.) This could, of course, be solved with a comment. But let's see if we can make things even easier on our future self.

In [10]:
def compute_p_ttest(t, n_obs, tail = 0):
    df = n_obs - 1
    if tail == 0:
        p = 2 * t_cdf(df, -np.abs(t))
    elif tail == 1:
        p = t_cdf(df, -t)
    elif tail == -1:
        p = t_cdf(df, t)
    return p

Now, we can compute the _p_-value for different types of test by changing the `tail` argument in the function above. We don't actually _use_ `tail = -1` in this analysis, but if you've already gone to the trouble of looking up these formulas, you might as well include it now. That way, you never have to think about those pesky formulas ever again! 

Okay, there's a better reason for thinking about use-cases that you aren't currently planning to avail yourself of. The more times the same code gets used, whether by you again or by someone else, the more likely it is that someone will catch a bug. So if you make you code _general_ -- that is, not limited in scope to your current analysis/project -- then it's more liklely that someone will come along one day and catch a bug. 

It might be confusing to someone what `tail = 1`, etc. means, so we'll add a _docstring_ to our function for posterity. A docstring should tell you what the inputs and outputs of a function are. If you write good docstrings, you shouldn't have to read the code within a function to tell how to use it. 

In [11]:
def compute_p_ttest(t, n_obs, tail = 0):
    '''
    Computes the p-value for a t-test.
    
    Arguments
    ---------
    t : float
        The t-statistic.
    n_obs : int
        The number of observations (i.e. sample size).
    tail : -1 or 0 or 1 (default = 0)
        If tail is 1, the alternative hypothesis is that the
        mean of the data is greater than 0 (upper tailed test).  If tail is 0,
        the alternative hypothesis is that the mean of the data is different
        than 0 (two tailed test).  If tail is -1, the alternative hypothesis
        is that the mean of the data is less than 0 (lower tailed test).
        
    Returns
    ---------
    p : float
        The p-value.
    '''
    df = n_obs - 1
    if tail == 0:
        p = 2 * t_cdf(df, -np.abs(t))
    elif tail == 1:
        p = t_cdf(df, -t)
    elif tail == -1:
        p = t_cdf(df, t)
    else:
        raise Exception('tail argument must be -1, 0, or 1!')
    return p

Since we have a docstring, you can now see the function documentation in a notebook by typing `?compute_p_ttest`. So that's useful. 

As a side note, you can format your docstrings anyway you want, but I recommend using the same format as above. Why? The conventional format is _machine readable_, which means that some software like [sphinx](https://www.sphinx-doc.org/en/master/) can use it to auto-generate HTML documentation for your code. 

Notice I also added a little bonus. Now, if you input a value of `tail` that isn't allowed, the function will give an informative error message, instead of just running and returning `None`. (Catching user errors in a function can save you a lot of debugging time later.)

Now, we can put our two functions together in a higher-level function to make our _t_-tests super easy.

In [12]:
def ttest_1samp(x, null_mean = 0, tail = 0):
    '''
    A one-sample t-test.
    
    Arguments
    ---------
    x : array of shape (n_observations,)
        Your data.
    null_mean : float
        The distribution mean under the null hypothesis.
    tail : -1 or 0 or 1 (default = 0)
        If tail is 1, the alternative hypothesis is that the
        mean of the data is greater than 0 (upper tailed test).  If tail is 0,
        the alternative hypothesis is that the mean of the data is different
        than 0 (two tailed test).  If tail is -1, the alternative hypothesis
        is that the mean of the data is less than 0 (lower tailed test).
        
    Returns
    ---------
    t : float
        The t-statistic.
    p : float
        The p-value.
    '''
    n_obs = x.shape[0]
    t = compute_t_1samp(x, null_mean)
    p = compute_p_ttest(t, n_obs, tail)
    return t, p
    
def ttest_paired(x0, x1, tail = 0):
    '''
    A paired-sample t-test.
    
    Arguments
    ---------
    x0 : array of shape (n_observations,)
    x1 : array of shape (n_observations,)
    tail : -1 or 0 or 1 (default = 0)
        If tail is 1, the alternative hypothesis is that the
        mean of x0 is greater than that of x1. If tail is -1, 
        the alternative is that the mean of x1 is greater than
        that of x0. If tail is 0, then the alternative is that
        the means are different from one another. 
        
    Returns
    ---------
    t : float
        The t-statistic.
    p : float
        The p-value.
    '''
    t, p = ttest_1samp(x0 - x1, 0, tail)
    return t, p

In [13]:
# our one-sample t-test
print('t = %f, p = %f'%ttest_1samp(sw, null_mean = 3, tail = 1))

t = 1.611015, p = 0.054646


In [14]:
# our paired -sample t-test
print('t = %f, p = %e'%ttest_paired(pl, pw, tail = 0))

t = 29.796754, p = 1.200807e-64


This is, I think, a big improvement over the original code. Notice that, as we increase the level of _abstraction_, what our code is doing becomes more and more clear. It is easy to tell, for instance, that a paired-sample test is just a one-sample test on the paired differences. This is accomplished without particularly detailed comments; it happens organically by using appropriate function names, which abstract away from the minor details of the program. If you use abstraction (such as functions) well, your main script should read a lot like the methods section of your paper. 

Speaking of which, while I have the functions sitting in this notebook for illustration, I don't actually like to keep my function definitions in the main script, if only because it keeps the main script less cluttered (again, because I like it to read sort of like a methods section -- a high level overview of the analysis). But also, there are some handy things you can do if you keep your functions in a seperate _module_.